In [1]:
import pandas as pd
import ollama
import re
import time

In [2]:
df = pd.read_csv('important_sentences.csv',index_col=0)

In [3]:
df.index.name = 'index'
df.rename(columns={'sentence': 'sentence1'}, inplace=True)
df['sentence2'] = ""  # Fill with empty strings
df['score'] = 0.0

In [9]:
df.columns

Index(['sentence1', 'sentence2', 'score'], dtype='object')

In [5]:
schrijfassistent_goed = """
FROM bramvanroy/geitje-7b-ultra:Q4_K_M

PARAMETER temperature 0.3
    
SYSTEM Bij elke zin die je ontvangt, verander je alleen de delen die niet overeenkomen met deze richtlijnen en geef je de gecorrigeerde zin terug als dit nodig is, zonder extra toelichting. Houd de lengte van de gecorrigeerde zin vergelijkbaar met de oorspronkelijke zin. Schrijf op een energieke en heldere manier. Schrijf op een enthousiaste manier zonder te overdrijven. Vermeld specifiek de datum, tijd, en kosten als deze zijn meegegeven, en gebruik toegankelijke en inclusieve taal. Houd de stijl direct en actief, en vermijd formeel of ambtelijk taalgebruik. Gebruik 'je' in informele communicatie zoals campagnes en sociale media, en 'u' in formele teksten zoals brieven en uitnodigingen.  Schrijf zonder jargon en wees je bewust van de diversiteit van lezers. Leg vaktermen uit als ze nodig zijn. Gebruik geen dialect of vreemde woorden tenzij relevant. Maak gebruik van eenvoudige en alledaagse woorden om verwarring te voorkomen. Schrijf genderneutraal en houd rekening met de inclusiviteit van personen met een beperking waar relevant. Gebruik de vorm '-aren' voor inwonersnamen zoals antwerpenaren. Schrijf getallen niet voluit voorbeeld: 10 ipv tien.
"""

stijlassistent_goed = """
FROM bramvanroy/geitje-7b-ultra:Q4_K_M

PARAMETER temperature 0.15
    
SYSTEM Bij elke zin die je ontvangt, verander je alleen de delen die niet overeenkomen met deze richtlijnen en geef je de gecorrigeerde zin terug als dit nodig is, zonder extra toelichting. Schrijf urls altijd met het www. protocol. Houd de lengte van de gecorrigeerde zin vergelijkbaar met de oorspronkelijke zin. Gebruik kleine letters voor 'stad', 'stadsdeel' en 'district'. RAAK URLS NIET AAN. Schrijf afdelingen en diensten binnen de stad met hoofdletters, behalve lidwoorden en voegwoorden. Gebruik 'we' alleen bij communicatie vanuit de redactie of om verbinding te tonen. Vermijd 'we' wanneer je spreekt namens een dienst of district. Gebruik dubbele aanhalingstekens voor citaten en enkele voor titels van werken zoals voorstellingen en boeken. Vermijd woorden uit andere talen. Zet ze cursief als dat noodzakelijk is. Gebruik de 24-uursnotatie. Schrijf getallen altijd als cijfers, met een punt voor duizendtallen en 'euro' voluit. Vermeld de volledige datum in tekst met de dag van de week indien mogelijk. Bij plaatsgebrek schrijf je compact met een slash.
""" 

ollama.create(model='schrijfassistent_goed', modelfile=schrijfassistent_goed)
ollama.create(model='stijlassistent_goed', modelfile=stijlassistent_goed)

{'status': 'success'}

In [6]:
# Function to clean URLs, modify time phrases, and remove non-alphanumeric characters in one pass
def clean_and_process_text(input_text):
    # Define regex patterns
    url_pattern = re.compile(r'\b(?:https?:\/\/)?(?:www\.)?([a-zA-Z0-9.-]+(?:\.[a-zA-Z]{2,}))\b')
    time_pattern = re.compile(r'\b(\d{1,2}):(\d{2})\s*uur\b')
    alphanumeric_pattern = re.compile(r'[^a-zA-Z0-9\s.,!?;:()\'"-\\\/‟„”$€¥£™©]')
    
    def replace_url(match):
        # Extract and format the domain
        domain = match.group(1)
        return f"www.{domain}" if not match.group(0).startswith('www.') else domain

    def replace_time(match):
        # Replace ':' with '.' for the time format
        return match.group(0).replace(':', '.')

    # Process the input text
    def process_text(text):
        # Replace URLs and modify time phrases
        text = url_pattern.sub(replace_url, text)
        text = time_pattern.sub(replace_time, text)
        # Remove non-alphanumeric characters
        return alphanumeric_pattern.sub('', text)
    
    # Return the processed text
    return process_text(input_text)

example_sentences = [
    "De vergadering vindt plaats op maandag 3 november om 10:00 uur en kost 15 euro. Meer info op https://antwerpen.be.",
]

# Start timing
start_time = time.time()

for original_sentence in example_sentences:
    rewritten_response = ollama.generate(model='schrijfassistent_goed', prompt=original_sentence)
    style_response = ollama.generate(model='stijlassistent_goed', prompt=rewritten_response['response'])

    final_output = clean_and_process_text(style_response['response'])

    print(f"Original: {original_sentence}")
    print(f"Final Output: {final_output}\n")

# End timing
end_time = time.time()

execution_time = end_time - start_time
print(f"Total execution time: {execution_time:.4f} seconds")

Original: De vergadering vindt plaats op maandag 3 november om 10:00 uur en kost 15 euro. Meer info op https://antwerpen.be.
Final Output: De vergadering is gepland voor maandag 3 november om 10.00 uur en de deelname kost €15. Voor meer informatie kun je terecht op www.antwerpen.be.

Total execution time: 4.0236 seconds


In [7]:
schrijfassistent_slecht = """
FROM bramvanroy/geitje-7b-ultra:Q4_K_M

PARAMETER temperature 1
    
SYSTEM Bij elke zin die je ontvangt, laat je alleen de delen die niet overeenkomen met deze richtlijnen intact en geef je de originele zin terug als dit niet nodig is, zonder extra uitleg. Houd de lengte van de originele zin niet noodzakelijk vergelijkbaar met de gecorrigeerde zin. Schrijf op een saaie en onduidelijke manier. Schrijf op een ongeïnspireerde manier zonder enige opwinding. Vermeld geen specifieke datum, tijd of kosten als deze zijn meegegeven, en gebruik taal die niet toegankelijk of exclusief is. Houd de stijl indirect en passief, en vermijd een directe communicatiestijl. Gebruik 'u' in informele communicatie zoals campagnes en sociale media, en 'je' in formele teksten zoals brieven en uitnodigingen. Schrijf vol jargon en negeer de diversiteit van lezers. Leg vaktermen nooit uit, ook al zijn ze nodig. Gebruik dialect en vreemde woorden, ongeacht de relevantie. Maak gebruik van complexe en ongebruikelijke woorden om verwarring te bevorderen. Schrijf niet genderneutraal en houd geen rekening met de inclusiviteit van personen met een beperking, zelfs als dat relevant is. Gebruik de vorm '-en' voor inwonersnamen. Schrijf getallen voluit, bijvoorbeeld: tien in plaats van 10."""

stijlassistent_slecht = """
FROM bramvanroy/geitje-7b-ultra:Q4_K_M

PARAMETER temperature 1
    
SYSTEM Bij elke zin die je ontvangt, verander je alleen de delen die overeenkomen met deze richtlijnen en geef je de gecorrigeerde zin terug als dit niet nodig is, zonder extra toelichting. Houd de lengte van de gecorrigeerde zin ongelijk aan de oorspronkelijke zin. Gebruik bij websites nooit 'www' als prefix. Gebruik hoofdletters voor 'stad', 'stadsdeel' en 'district'. Schrijf afdelingen en diensten binnen de stad met kleine letters, inclusief lidwoorden en voegwoorden. Gebruik 'we' altijd bij communicatie vanuit de redactie of om verbinding te tonen. Vermijd 'we' nooit wanneer je spreekt namens een dienst of district. Gebruik enkele aanhalingstekens voor citaten en dubbele voor titels van werken zoals voorstellingen en boeken. Gebruik woorden uit andere talen zonder cursivering. Gebruik de 12-uursnotatie. Schrijf getallen altijd als woorden, met een komma voor duizendtallen en '€' als symbool. Vermeld de datum in tekst zonder de dag van de week. Bij plaatsgebrek schrijf je ruim met een spatie.
""" 

ollama.create(model='schrijfassistent_slecht', modelfile=schrijfassistent_goed)
ollama.create(model='stijlassistent_slecht', modelfile=stijlassistent_goed)

{'status': 'success'}

In [8]:
example_sentences = [
    "De vergadering vindt plaats op maandag 3 november om 10:00 uur en kost 15 euro. Meer info op https://antwerpen.be.",
]

# Start timing
start_time = time.time()

for original_sentence in example_sentences:
    rewritten_response = ollama.generate(model='schrijfassistent_slecht', prompt=original_sentence)
    style_response = ollama.generate(model='stijlassistent_slecht', prompt=rewritten_response['response'])

    final_output = style_response['response']
    print(f"Original: {original_sentence}")
    print(f"Final Output: {final_output}\n")

# End timing
end_time = time.time()

execution_time = end_time - start_time
print(f"Total execution time: {execution_time:.4f} seconds")

Original: De vergadering vindt plaats op maandag 3 november om 10:00 uur en kost 15 euro. Meer info op https://antwerpen.be.
Final Output: Op maandag 3 november, vanaf 10:00 uur, is er een vergadering die je niet wilt missen! De kosten zijn €15 en voor meer informatie kun je terecht op https://antwerpen.be.

Total execution time: 2.9754 seconds


In [14]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(subject, body, to_email):
    # Email account credentials
    from_email = "jurrean.denys@student.ap.be"  # Replace with your Outlook email
    from_password = "B0bd3z0mbie!1234"  # Replace with your password or app password

    # Create the email
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    # Attach the email body
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Set up the SMTP server for Outlook
        with smtplib.SMTP('smtp.office365.com', 587) as server:  # For Outlook/Office 365
            server.starttls()  # Upgrade the connection to a secure encrypted SSL/TLS
            server.login(from_email, from_password)
            server.send_message(msg)

        print("Email sent successfully!")

    except Exception as e:
        print(f"Failed to send email: {e}")

# Usage example after processing the DataFrame
if __name__ == "__main__":
    # After your DataFrame processing code...
    
    # Notify via email
    send_email(
        subject="Data Processing Complete",
        body="The DataFrame processing is complete. Please check the output file.",
        to_email="jurreandenys@gmail.com"  # Replace with your recipient email
    )

Failed to send email: (535, b'5.7.139 Authentication unsuccessful, account has an issue. Contact your administrator. [FR3P281CA0100.DEUP281.PROD.OUTLOOK.COM 2024-11-03T15:45:05.291Z 08DCFB25EF9852C3]')
